In [ ]:
import os
import pandas as pd
import numpy as np
from osgeo import gdal
import georasters as gr
import rasterio
from rasterio.mask import mask
import py7zr
import subprocess
import geopandas as gpd
import zipfile

## Clipping to Sweden (or Stockholm County)

In [2]:
def clip_raster(raster_path, boundary, output_folder="clipped_GHS_Data"):
    # create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, sweden_boundary.geometry, crop=True)
        out_meta = src.meta.copy()
        
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        output_path = os.path.join(output_folder, f"clipped_{os.path.basename(raster_path)}")
        
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)
            
    print(f"Clipped raster saved to {output_path}")

In [ ]:
# original raster paths
raster_paths = [''] # GHS raster paths

# Sweden shapefile as GeoDataFrame
sweden_boundary = gpd.read_file('Sweden_shapefile/se.json')

# Get Stockholm county boundaries
se_counties = gpd.read_file('Sweden_shapefile/se_counties.json')
stockholm_boundary = se_counties.loc[[9]]

# check CRS of raster
with rasterio.open(ghs1) as src:
    raster_crs = src.crs
    print(f"Raster CRS: {raster_crs}")

# check CRS of shapefile
print(f"Sweden Boundary CRS: {sweden_boundary.crs}")

# transform the CRS of the Sweden / Stockholm boundary to match the raster CRS if they don't match
if sweden_boundary.crs != raster_crs:
    sweden_boundary = sweden_boundary.to_crs(raster_crs)
    print(f'Transformed Sweden CRS to match raster CRS')

In [ ]:
# clip the rasters
for path in raster_paths:
    clip_raster(path, sweden_boundary)

# Merge Raster Tiles for each GHS Feature

In [36]:
from rasterio.merge import merge

def merge_rasters(raster_paths):
    src_files_to_mosaic = []
    for path in raster_paths:
        src = rasterio.open(path)
        src_files_to_mosaic.append(src)

    mosaic, out_trans = merge(src_files_to_mosaic)

    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": src.crs
    })
    
    for src in src_files_to_mosaic:
        src.close()

    return mosaic, out_meta

In [ ]:
# paths to clipped GHS data
pop_paths = ['clipped_GHS_Data\clipped_GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R2_C19.tif',
            'clipped_GHS_Data\clipped_GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R2_C20.tif',
            'clipped_GHS_Data\clipped_GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R3_C19.tif',
            'clipped_GHS_Data\clipped_GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R3_C20.tif']

surface_paths = ['clipped_GHS_Data/clipped_GHS_BUILT_S_E2030_GLOBE_R2023A_54009_100_V1_0_R2_C19.tif',
                'clipped_GHS_Data/clipped_GHS_BUILT_S_E2030_GLOBE_R2023A_54009_100_V1_0_R2_C20.tif',
                'clipped_GHS_Data/clipped_GHS_BUILT_S_E2030_GLOBE_R2023A_54009_100_V1_0_R3_C19.tif',
                'clipped_GHS_Data/clipped_GHS_BUILT_S_E2030_GLOBE_R2023A_54009_100_V1_0_R3_C20.tif']

height_paths = ['clipped_GHS_Data\clipped_GHS_BUILT_H_AGBH_E2018_GLOBE_R2023A_54009_100_V1_0_R2_C19.tif',
                'clipped_GHS_Data\clipped_GHS_BUILT_H_AGBH_E2018_GLOBE_R2023A_54009_100_V1_0_R2_C20.tif',
                'clipped_GHS_Data\clipped_GHS_BUILT_H_AGBH_E2018_GLOBE_R2023A_54009_100_V1_0_R3_C19.tif',
                'clipped_GHS_Data\clipped_GHS_BUILT_H_AGBH_E2018_GLOBE_R2023A_54009_100_V1_0_R3_C20.tif']

land_paths = ['clipped_GHS_Data\clipped_GHS_LAND_E2018_GLOBE_R2022A_54009_100_V1_0_R2_C19.tif',
             'clipped_GHS_Data\clipped_GHS_LAND_E2018_GLOBE_R2022A_54009_100_V1_0_R2_C20.tif',
             'clipped_GHS_Data\clipped_GHS_LAND_E2018_GLOBE_R2022A_54009_100_V1_0_R3_C19.tif',
             'clipped_GHS_Data\clipped_GHS_LAND_E2018_GLOBE_R2022A_54009_100_V1_0_R3_C20.tif']

volume_paths = ['clipped_GHS_Data\clipped_GHS_BUILT_V_E2020_GLOBE_R2023A_54009_100_V1_0_R2_C19.tif',
             'clipped_GHS_Data\clipped_GHS_BUILT_V_E2020_GLOBE_R2023A_54009_100_V1_0_R2_C20.tif',
             'clipped_GHS_Data\clipped_GHS_BUILT_V_E2020_GLOBE_R2023A_54009_100_V1_0_R3_C19.tif',
             'clipped_GHS_Data\clipped_GHS_BUILT_V_E2020_GLOBE_R2023A_54009_100_V1_0_R3_C20.tif']


# merge the "sub"-rasters into one for each GHS value
mosaic1, meta1 = merge_rasters(pop_paths)
mosaic2, meta2 = merge_rasters(surface_paths)
mosaic3, meta3 = merge_rasters(height_paths)
mosaic4, meta4 = merge_rasters(land_paths)
mosaic5, meta5 = merge_rasters(volume_paths)

In [ ]:
def save_raster(mosaic, meta, output_path):
    with rasterio.open(output_path, 'w', **meta) as dest:
        dest.write(mosaic)
        
save_raster(mosaic1, meta1, 'POP_merged.tif')
save_raster(mosaic2, meta2, 'SURFACE_merged.tif')
save_raster(mosaic3, meta3, 'HEIGHT_merged.tif')
save_raster(mosaic4, meta4, 'LAND_merged.tif')
save_raster(mosaic5, meta5, 'VOL_merged.tif')

# Convert Rasters to DataFrame and Merge

In [40]:
def raster_to_dataframe(mosaic, transform):
    rows, cols = np.indices(mosaic.shape)
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    df = pd.DataFrame({
        'row': rows.flatten(),
        'col': cols.flatten(),
        'x': np.array(xs).flatten(),
        'y': np.array(ys).flatten(),
        'value': mosaic.flatten()
    })
    return df

POP_df = raster_to_dataframe(mosaic1[0], meta1['transform'])
SURFACE_df = raster_to_dataframe(mosaic2[0], meta2['transform'])
HEIGHT_df = raster_to_dataframe(mosaic3[0], meta3['transform'])
LAND_df = raster_to_dataframe(mosaic4[0], meta4['transform'])
VOL_df = raster_to_dataframe(mosaic5[0], meta5['transform'])

In [50]:
merged_df = POP_df.merge(SURFACE_df, on=['row', 'col', 'x', 'y'], how='outer', suffixes=('_POP', '_SURFACE'))
merged_df = merged_df.merge(HEIGHT_df, on=['row', 'col', 'x', 'y'], how='outer', suffixes=('', '_HEIGHT'))
merged_df = merged_df.merge(LAND_df, on=['row', 'col', 'x', 'y'], how='outer', suffixes=('', '_LAND'))
merged_df = merged_df.merge(VOL_df, on=['row', 'col', 'x', 'y'], how='outer', suffixes=('', '_VOL'))

# rename cols
merged_df = merged_df.rename(columns={'value_POP': 'POP',
                                      'value_SURFACE': 'SURFACE',
                                      'value': 'HEIGHT',
                                      'value_LAND': 'LAND',
                                     'value_VOL': 'VOLUME'})

In [55]:
# Remove rows with values indicating NoData
ghs_df = merged_df[(buffer_joined['LAND'] != 65535) & (buffer_joined['SURFACE'] != 65535) & 
                            (buffer_joined['VOLUME'] != 4294967295) & (buffer_joined['POP'] != -200) &
                            (buffer_joined['HEIGHT'] != -1.0)]

In [56]:
# save the stockholm_df as csv
ghs_df.to_csv('GHS_100x1.csv')

# Plot Rasters

In [ ]:
from rasterio.plot import show

raster_path = 'POP_merged.tif' # example with POPULATION

with rasterio.open(raster_path) as src:
    # Read the raster data
    raster_data = src.read(1)
    
    # plot the raster data using imshow
    plt.figure(figsize=(10, 10))
    im = plt.imshow(raster_data, cmap='viridis')
    plt.title('Raster Visualization')
    plt.xlabel('Column #')
    plt.ylabel('Row #')
    
    # add colorbar
    cbar = plt.colorbar(im, label='Value')
    
    plt.show()

# Spatial Join with SE Data
(example is for all of Sweden)

In [ ]:
from shapely.geometry import Point
from scipy.spatial import cKDTree

In [ ]:
# read Sweden df
se_df = pd.read_csv('sweden_df.csv')
se_df['bLabel'] = se_df['label'].apply(lambda x: 1 if x == "Labeled" else 0) # add numeric labeling
se_gdf = gpd.GeoDataFrame(se_df, geometry=gpd.points_from_xy(se_df.lon, se_df.lat)) # convert to GeoDataFrame

# read GHS value data
ghs_df = pd.read_csv('GHS_merged_v2.csv')

In [ ]:
se_gdf.crs = "EPSG:4326"

# Reproject to the same CRS as raster dat
se_gdf = se_gdf.to_crs("ESRI:54009")

geometry = [Point(xy) for xy in zip(ghs_df['x'], ghs_df['y'])]
ghs_gdf = gpd.GeoDataFrame(ghs_df, geometry=geometry)
ghs_gdf.crs = "ESRI:54009"

In [ ]:
se_gdf['x'] = se_gdf.geometry.x
se_gdf['y'] = se_gdf.geometry.y

# Create cKDTree
tree = cKDTree(ghs_gdf[['x', 'y']])

# Query the tree for the nearest neighbors within a specified distance threshold
dist, idx = tree.query(se_gdf[['x', 'y']], k=1, distance_upper_bound=70.71068)

# idx contains the indices of the nearest neighbors in raster_gdf
# dist contains the distances to the nearest neighbors

# Filter out points that are beyond the distance threshold
valid = dist < np.inf

# Assign the nearest raster values to SE_gdf
se_gdf.loc[valid, 'POP'] = ghs_gdf.iloc[idx[valid]]['POP'].values
se_gdf.loc[valid, 'SURFACE'] = ghs_gdf.iloc[idx[valid]]['SURFACE'].values
se_gdf.loc[valid, 'HEIGHT'] = ghs_gdf.iloc[idx[valid]]['HEIGHT'].values
se_gdf.loc[valid, 'LAND'] = ghs_gdf.iloc[idx[valid]]['LAND'].values
se_gdf.loc[valid, 'VOLUME'] = ghs_gdf.iloc[idx[valid]]['VOLUME'].values

# Fill any missing values with NaN
se_gdf.fillna(np.nan, inplace=True)

# Convert back to a regular DataFrame and drop NaN
df_merged = pd.DataFrame(se_gdf.drop(columns='geometry')).dropna()

# Save df
df_merged.to_csv('spatialjoined70_v2.csv')